In [9]:
import numpy as np
import pandas as pd
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.preprocessing import StandardScaler


In [10]:
Car = pd.read_csv('car.data', sep=",")
Car.columns = [
    "buying_price",           # vhigh
    "maintenance_price",      # vhigh.1
    "doors",                  # 2
    "persons",                # 2.1
    "luggage_boot_size",      # small
    "safety",                 # low
    "car_acceptability"       # unacc
]


## Data Cleaning

In [14]:
Car['car_acceptability'] = Car['car_acceptability'].replace({
    'unacc': 0,  # Unacceptable
    'acc': 1,    # Acceptable
    'good': 1,   # Acceptable
    'vgood': 1  # Acceptable
})


C:\Users\akash\AppData\Local\Temp\ipykernel_25336\1832253008.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  Car['car_acceptability'] = Car['car_acceptability'].replace({


In [16]:
Car['car_acceptability'].value_counts()

car_acceptability
0    1209
1     518
Name: count, dtype: int64

In [17]:
label_encoders = {}
for column in Car.columns:
    if Car[column].dtype == 'object':  # Check if column is categorical
        le = LabelEncoder()
        Car[column] = le.fit_transform(Car[column])
        label_encoders[column] = le  # Store the encoder for future reference

# 3. Verify encoding
print("\nEncoded Data (first 5 rows):")
print(Car.head())



Encoded Data (first 5 rows):
   buying_price  maintenance_price  doors  persons  luggage_boot_size  safety  \
0             3                  3      0        0                  2       2   
1             3                  3      0        0                  2       0   
2             3                  3      0        0                  1       1   
3             3                  3      0        0                  1       2   
4             3                  3      0        0                  1       0   

   car_acceptability  
0                  0  
1                  0  
2                  0  
3                  0  
4                  0  


In [18]:
Car

,buying_price,maintenance_price,doors,persons,luggage_boot_size,safety,car_acceptability
0,3,3,0,0,2,2,0
1,3,3,0,0,2,0,0
2,3,3,0,0,1,1,0
3,3,3,0,0,1,2,0
4,3,3,0,0,1,0,0
...,...,...,...,...,...,...,...
1722,1,1,3,2,1,2,1
1723,1,1,3,2,1,0,1
1724,1,1,3,2,0,1,0
1725,1,1,3,2,0,2,1


## Now onto Using ML Models